In [1]:
import os
import random

os.listdir('../input/')

In [4]:
### read words ###
text_file = open('../input/input-data/words_250000_train.txt',"r")
full_dictionary = text_file.read().splitlines()

In [5]:
import numpy as np

random.seed(40)

import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

random.seed(40)

# Sample data for demonstration
words = full_dictionary

def generate_training_data_3(words):
    X = []
    y = []
    for word in words:
      # print(word)
      for char in np.unique(list(word)):
          X.append(word.replace(char,"_"))  # Input with missing character
          y.append(char)
          try:
            X.append(word.replace(char,"_").replace(random.choice(list(word.replace(char,""))),"_"))  # Input with missing character
            y.append(char)                      # Target character (the missing one)
          except:
            pass
          # print(word.replace(char,"_"))
    return X, y


# Toy dataset
X_train_3, y_train_3 = generate_training_data_3(words)

words_3 = X_train_3
labels_3 = y_train_3

max_length_3 = max(len(word) for word in words_3)

# Create a dictionary for character to index mapping
char_to_index = {char: idx for idx, char in enumerate(set(''.join(words_3)))}
index_to_char = {idx: char for char, idx in char_to_index.items()}

# Convert words and labels to numerical format
X_padded_3 = pad_sequences([[char_to_index[char] for char in word] for word in words_3], maxlen=max_length_3, padding='post')
y_padded_3 = np.array([char_to_index[label] for label in labels_3])

X_train_3, X_temp_3, y_train_3, y_temp_3 = train_test_split(X_padded_3, y_padded_3, test_size=0.3, random_state=42)
X_val_3, X_test_3, y_val_3, y_test_3 = train_test_split(X_temp_3, y_temp_3, test_size=0.5, random_state=42)

# print("Padded data:")
# for i in range(len(words)):
#     print(f"{words[i]} -> {labels[i]}")

# print("\nPadded and numerical data:")
# print(X_padded_2)
# print(y_padded_2)


2024-06-23 09:11:58.030390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 09:11:58.030517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 09:11:58.155272: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# BiLSTM with attention

## attention 

## LSTM 256

## embedding - 50

## no dropout

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

# Define a checkpoint callback
checkpoint_callback_6 = ModelCheckpoint('best_model_6.keras', 
                                      monitor='val_loss', 
                                      save_best_only=True, 
                                      mode='min', 
                                      verbose=1)

early_stopping_callback = EarlyStopping(monitor='val_loss', 
                                       patience=3, 
                                       mode='min', 
                                       verbose=1)

In [ ]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# # instantiate a distribution strategy
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Concatenate, Layer
from tensorflow.keras import Input, Model
import keras
# Define parameters
vocab_size = len(char_to_index)  # Size of vocabulary (unique characters)
embedding_dim = 50  # Dimension of character embeddings
lstm_units = 256  # Number of units in LSTM layers
num_layers = 3  # Number of BiLSTM layers
dropout_rate = 0.1  # Dropout rate

# Custom Attention Layer
@keras.saving.register_keras_serializable()
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")

        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = tf.squeeze(tf.tanh(tf.matmul(x, self.W) + self.b), axis=-1)
        at = tf.nn.softmax(et)
        at = tf.expand_dims(at, axis=-1)
        output = x * at
        return tf.reduce_sum(output, axis=1)

# Define Sequential model
model_6 = Sequential()

# Add layers to the model
model_6.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length_3))

# Add BiLSTM layers with Dropout
for _ in range(num_layers):
    model_6.add(Bidirectional(LSTM(units=lstm_units, return_sequences=True)))
    model_6.add(Dropout(rate=dropout_rate))

# Add Attention Layer
model_6.add(AttentionLayer())

# Output layer
model_6.add(Dense(units=vocab_size, activation='softmax'))

# Compile the model
model_6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_6.summary()


In [9]:
# Train the model
model_6.fit( X_train_3, 
            y_train_3, 
            epochs=3, 
            batch_size=256, 
            validation_data=(X_val_3, y_val_3), 
            verbose=1,callbacks=[checkpoint_callback_6,early_stopping_callback])

Epoch 1/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.3576 - loss: 2.0250
Epoch 1: val_loss improved from inf to 1.46096, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 628s 67ms/step - accuracy: 0.3576 - loss: 2.0249 - val_accuracy: 0.5023 - val_loss: 1.4610
Epoch 2/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5176 - loss: 1.4068
Epoch 2: val_loss improved from 1.46096 to 1.34231, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 621s 68ms/step - accuracy: 0.5176 - loss: 1.4068 - val_accuracy: 0.5338 - val_loss: 1.3423
Epoch 3/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5556 - loss: 1.2739
Epoch 3: val_loss improved from 1.34231 to 1.29647, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 618s 67ms/step - accuracy: 0.5556 - loss: 1.2739 - val_accuracy: 0.5456 - val_loss: 1.2965


In [12]:
# Train the model
model_6.fit( X_train_3, 
            y_train_3, 
            epochs=7, 
            batch_size=256, 
            validation_data=(X_val_3, y_val_3), 
            verbose=1,callbacks=[checkpoint_callback_6,early_stopping_callback])

Epoch 1/7
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5770 - loss: 1.2009
Epoch 1: val_loss improved from 1.29647 to 1.26940, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 618s 67ms/step - accuracy: 0.5770 - loss: 1.2009 - val_accuracy: 0.5522 - val_loss: 1.2694
Epoch 2/7
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5918 - loss: 1.1516
Epoch 2: val_loss improved from 1.26940 to 1.26491, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 617s 67ms/step - accuracy: 0.5918 - loss: 1.1516 - val_accuracy: 0.5558 - val_loss: 1.2649
Epoch 3/7
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.6027 - loss: 1.1156
Epoch 3: val_loss improved from 1.26491 to 1.25808, saving model to best_model_6.keras
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 618s 67ms/step - accuracy: 0.6027 - loss: 1.1156 - val_accuracy: 0.5575 - val_loss: 1.2581
Epoch 4/7
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.6109 - loss: 1.0910
Epoch 4: val_loss imp

In [66]:
# checkpoint_callback_total6 = ModelCheckpoint('best_model_6.keras', 
#                                       monitor='val_loss', 
#                                       save_best_only=True, 
#                                       mode='min', 
#                                       verbose=1)

In [13]:
# Train the model
model_6.fit(X_train_3, 
            y_train_3, 
            epochs=3,      
            batch_size=256,                     
            validation_data=(X_val_3, y_val_3), 
            verbose=1,
            callbacks=[checkpoint_callback_6,early_stopping_callback])

Epoch 1/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6297 - loss: 1.0308
Epoch 1: val_loss did not improve from 1.24678
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 622s 68ms/step - accuracy: 0.6297 - loss: 1.0308 - val_accuracy: 0.5604 - val_loss: 1.2492
Epoch 2/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6336 - loss: 1.0203
Epoch 2: val_loss did not improve from 1.24678
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 629s 68ms/step - accuracy: 0.6336 - loss: 1.0203 - val_accuracy: 0.5600 - val_loss: 1.2582
Epoch 3/3
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6364 - loss: 1.0107
Epoch 3: val_loss did not improve from 1.24678
9194/9194 ━━━━━━━━━━━━━━━━━━━━ 621s 67ms/step - accuracy: 0.6364 - loss: 1.0107 - val_accuracy: 0.5600 - val_loss: 1.2575
Epoch 3: early stopping


# Complete Training data
# randome shuffling 

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Concatenate, Layer
from tensorflow.keras import Input, Model
import keras
# Define parameters
vocab_size = len(char_to_index)  # Size of vocabulary (unique characters)
embedding_dim = 50  # Dimension of character embeddings
lstm_units = 256  # Number of units in LSTM layers
num_layers = 3  # Number of BiLSTM layers
dropout_rate = 0.1  # Dropout rate

# Custom Attention Layer
@keras.saving.register_keras_serializable()
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")

        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = tf.squeeze(tf.tanh(tf.matmul(x, self.W) + self.b), axis=-1)
        at = tf.nn.softmax(et)
        at = tf.expand_dims(at, axis=-1)
        output = x * at
        return tf.reduce_sum(output, axis=1)

# Define Sequential model
model_complete_6 = Sequential()

# Add layers to the model
model_complete_6.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length_3))

# Add BiLSTM layers with Dropout
for _ in range(num_layers):
    model_complete_6.add(Bidirectional(LSTM(units=lstm_units, return_sequences=True)))
    model_complete_6.add(Dropout(rate=dropout_rate))

# Add Attention Layer
model_complete_6.add(AttentionLayer())

# Output layer
model_complete_6.add(Dense(units=vocab_size, activation='softmax'))

# Compile the model
model_complete_6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_complete_6.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_7 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_8 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer_2               │ ?                      │   0 (unbuilt) │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
from sklearn.utils import shuffle

X_train_complete_3, y_train_complete_3 = shuffle( X_padded_3, y_padded_3 )

# Train the model
model_complete_6.fit( X_train_complete_3, 
            y_train_complete_3,
            epochs=8, 
            batch_size=256, 
#             validation_data=(X_val_3, y_val_3), 
            verbose=1,
#             callbacks=[checkpoint_callback_6,early_stopping_callback]
           )

Epoch 1/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 830s 63ms/step - accuracy: 0.3843 - loss: 1.9180
Epoch 2/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 823s 63ms/step - accuracy: 0.5356 - loss: 1.3418
Epoch 3/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 824s 63ms/step - accuracy: 0.5648 - loss: 1.2346
Epoch 4/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 827s 63ms/step - accuracy: 0.5810 - loss: 1.1759
Epoch 5/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 821s 63ms/step - accuracy: 0.5912 - loss: 1.1411
Epoch 6/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 830s 63ms/step - accuracy: 0.5983 - loss: 1.1167
Epoch 7/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 831s 63ms/step - accuracy: 0.6037 - loss: 1.0979
Epoch 8/8
13135/13135 ━━━━━━━━━━━━━━━━━━━━ 833s 63ms/step - accuracy: 0.6080 - loss: 1.0840


In [10]:
model_complete_6.save('best_model_6_complete.keras')

In [ ]:
model_complete_6_recon = tf.keras.models.load_model('best_model_6_complete.keras')

In [13]:
# model_complete_6_recon = tf.keras.models.load_model('best_model_6_complete.keras')

In [15]:
## load model 
# model_6_recon = tf.keras.models.load_model('best_model_6.keras')

In [27]:
# Evaluate on test set
loss, accuracy = model_complete_6_recon.evaluate(X_test_3, y_test_3, verbose=1)
print(f"validation set Accuracy: {accuracy*100:.2f}%")

15762/15762 ━━━━━━━━━━━━━━━━━━━━ 137s 9ms/step - accuracy: 0.6346 - loss: 1.0046
validation set Accuracy: 63.64%


In [60]:
# Evaluate on test set
loss, accuracy = model_6_recon.evaluate(X_test_3, y_test_3, verbose=1)
print(f"validation set Accuracy: {accuracy*100:.2f}%")

15762/15762 ━━━━━━━━━━━━━━━━━━━━ 135s 8ms/step - accuracy: 0.5564 - loss: 1.2554
validation set Accuracy: 55.77%
